<a href="https://colab.research.google.com/github/orodriguezq/orodriguezq-Senales_Y_Sistemas/blob/main/Parcial_2_SyS_2025-1/Correccion_Punto2_Parcial2_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# ── Instalación de librerías para la aplicación ──────────────────────────
!pip install streamlit librosa --quiet
                                     # Streamlit: interfaz web
                                     # Librosa: carga y análisis de audio

# ── Descarga y configuración de Cloudflared ─────────────────────────────
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
                                     # Descarga la última versión del binario
!chmod +x cloudflared                # Asigna permiso de ejecución al archivo
!mv cloudflared /usr/local/bin/cloudflared
                                     # Mueve el binario a una carpeta del PATH
!pip install streamlit scipy soundfile matplotlib pytube pydub ffmpeg-python
!apt-get install -y ffmpeg
!pip install --upgrade pytube

!pip install yt-dlp
!pip install streamlit scipy soundfile matplotlib pydub ffmpeg-python
!apt-get install -y ffmpeg


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 42.6 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [26]:
%%writefile ssb_dashboard.py
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import hilbert, butter, lfilter, freqz
from scipy.fft import fft, fftfreq
import soundfile as sf
from pytube import YouTube

from pydub import AudioSegment
import yt_dlp
import tempfile
import os
import re


# --- FUNCIONES AUXILIARES ---

def download_youtube_audio_segment(youtube_url_or_id, start_sec=0, duration_sec=5, fs_target=8000):
    # Extrae el ID si es URL
    m = re.search(r"([A-Za-z0-9_-]{11})", youtube_url_or_id)
    if not m:
        raise ValueError("No se pudo extraer el ID de YouTube.")
    video_id = m.group(1)
    url = f"https://www.youtube.com/watch?v={video_id}"
    # Configuración para yt-dlp
    outtmpl = "yt_audio.%(ext)s"
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': outtmpl,
        'quiet': True,
        'noplaylist': True
    }
    # Descarga con yt-dlp
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)
        ext = info.get('ext', 'm4a')
        fname = f"yt_audio.{ext}"
    # Lee y recorta con pydub
    audio = AudioSegment.from_file(fname).set_channels(1).set_frame_rate(fs_target)
    segment = audio[start_sec*1000:(start_sec+duration_sec)*1000]
    tmp_wav = tempfile.NamedTemporaryFile(suffix=".wav", delete=False)
    segment.export(tmp_wav.name, format="wav")
    import soundfile as sf
    x, fs = sf.read(tmp_wav.name)
    os.remove(fname)
    os.remove(tmp_wav.name)
    return x, fs

def plot_time_freq(signal, title, fs, desc=""):
    fig, axs = plt.subplots(1,2, figsize=(8,3))
    axs[0].plot(np.arange(len(signal))/fs, signal)
    axs[0].set_title(title + " (tiempo)")
    axs[0].set_xlabel("Tiempo [s]")
    axs[1].plot(fftfreq(len(signal), 1/fs), np.abs(fft(signal)))
    axs[1].set_xlim(0, fs/2)
    axs[1].set_title(title + " (espectro)")
    axs[1].set_xlabel("Frecuencia [Hz]")
    st.pyplot(fig)
    if desc:
        st.markdown(f"**Descripción:** {desc}")

# --- DASHBOARD ---

st.title("Dashboard SSB-AM: Modulación y Demodulación")

# Parámetros básicos
tipo_mensaje = st.selectbox("Tipo de mensaje", ["Pulso rectangular", "Segmento de canción de YouTube"])
fs = 8000
t = np.linspace(0, 1, fs, endpoint=False)
fc = st.slider("Frecuencia de portadora (Hz)", min_value=100, max_value=2000, value=800)

# Entrada de mensaje
if tipo_mensaje == "Pulso rectangular":
    width = st.slider("Ancho del pulso (%)", 1, 99, 20)
    mensaje = np.zeros_like(t)
    mensaje[:int(width/100*len(t))] = 1
    mensaje = mensaje.astype(float)
else:
    url = st.text_input("Pega el enlace de YouTube aquí:")
    start = st.number_input("¿Desde qué segundo quieres extraer los 5 segundos?", 0, 300, 0)
    msg_ok = False
    if url:
        try:
            mensaje, fs = download_youtube_audio_segment(url, int(start), duration_sec=5, fs_target=8000)
            if mensaje.ndim > 1:
                mensaje = mensaje[:,0]
            t = np.linspace(0, len(mensaje)/fs, len(mensaje), endpoint=False)
            msg_ok = True
        except Exception as e:
            st.warning(f"⚠️ No se pudo procesar el audio de YouTube. Intenta con otro enlace, o sube un archivo WAV (máx. 5 segundos, 1 canal, 8000 Hz).\n\nError: {e}")
            st.info("Puedes subir un archivo de audio .wav a continuación:")

    if not msg_ok:
        archivo = st.file_uploader("Carga un archivo .wav (≤5 s, mono, 8kHz)", type=["wav"])
        if archivo is not None:
            mensaje, fs = sf.read(archivo)
            if mensaje.ndim > 1:
                mensaje = mensaje[:,0]
            t = np.linspace(0, len(mensaje)/fs, len(mensaje), endpoint=False)
            msg_ok = True

    if not msg_ok:
        st.stop()



# Hilbert y SSB
lado = st.radio("Banda", ["USB (Superior)", "LSB (Inferior)"])
mh = np.imag(hilbert(mensaje))
if lado.startswith("USB"):
    ssb = mensaje*np.cos(2*np.pi*fc*t) - mh*np.sin(2*np.pi*fc*t)
else:
    ssb = mensaje*np.cos(2*np.pi*fc*t) + mh*np.sin(2*np.pi*fc*t)

# Demodulación coherente
demod = 2 * ssb * np.cos(2*np.pi*fc*t)
N, Wn = 4, 0.2  # Orden y freq normalizada del filtro
b, a = butter(N, Wn)
rec = lfilter(b, a, demod)

# --- GRAFICAS Y DESCRIPCIONES ---
plot_time_freq(
    mensaje, "Mensaje", fs,
    "Esta es la señal mensaje elegida, en el tiempo y frecuencia."
)
plot_time_freq(
    ssb, "SSB Modulada", fs,
    "Señal modulada en banda lateral única (SSB), usando la transformada de Hilbert para eliminar una banda lateral."
)
plot_time_freq(
    demod, "Señal Demodulada (sin filtrar)", fs,
    "Multiplicación de la SSB con la portadora; aparecen componentes a baja y alta frecuencia."
)
plot_time_freq(
    rec, "Señal Recuperada", fs,
    "Resultado de filtrar la señal demodulada con un filtro digital IIR pasa-bajos, recuperando la señal mensaje."
)

# --- FILTRO: BODE Y POLOS-CEROS ---
st.header("Filtro Pasa-Bajos Digital (IIR)")

# Bode
w, h = freqz(b, a, worN=8000)
fig, (ax1, ax2) = plt.subplots(2,1, figsize=(7,5))
ax1.plot(0.5*fs*w/np.pi, 20*np.log10(abs(h)))
ax1.set_title('Bode Magnitud')
ax1.set_ylabel('dB')
ax2.plot(0.5*fs*w/np.pi, np.angle(h))
ax2.set_title('Bode Fase')
ax2.set_ylabel('radianes')
ax2.set_xlabel('Hz')
plt.tight_layout()
st.pyplot(fig)
st.markdown("**Descripción:** Diagrama de Bode (magnitud y fase) del filtro digital IIR utilizado en la etapa de demodulación.")

# Polos y ceros estilo notebook RLC
z = np.roots(b)
p = np.roots(a)
fig, ax = plt.subplots()
ax.scatter(np.real(z), np.imag(z), marker='o', label='Ceros', color='blue')
ax.scatter(np.real(p), np.imag(p), marker='x', label='Polos', color='red')
ax.axhline(0, color='gray', lw=1)
ax.axvline(0, color='gray', lw=1)
ax.set_xlabel('Parte real')
ax.set_ylabel('Parte imaginaria')
ax.set_title('Plano de polos y ceros (Filtro IIR)')
ax.legend()
st.pyplot(fig)
st.markdown("**Descripción:** Ubicación de los polos (x) y ceros (o) del filtro digital IIR en el plano complejo.")

st.markdown("""
---
**Etapas del proceso:**
1. Selecciona un mensaje (pulso rectangular o segmento de canción de YouTube).
2. Observa su modulación SSB (banda lateral única).
3. Ve el efecto de la demodulación y filtrado en tiempo y frecuencia.
4. Analiza el filtro IIR mediante su respuesta en frecuencia (Bode) y sus polos y ceros.
""")


Overwriting ssb_dashboard.py


In [27]:
# 🟢 Celda de código 3: Ejecución de la Aplicación y Exposición Pública

import time, re

# 0) Instala cloudflared si no está disponible
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 1) Cerrar instancias antiguas de Streamlit y Cloudflared
!pkill -f streamlit
!pkill -f cloudflared

# 2) Iniciar la app Streamlit en background (sin mostrar logs)
!streamlit run ssb_dashboard.py &> /dev/null &

# 3) Esperar unos segundos para que Streamlit arranque correctamente
time.sleep(3)

# 4) Lanzar el túnel con el binario oficial de Cloudflared
!cloudflared tunnel --url http://localhost:8501 > cloudflared.log 2>&1 &

# 5) Dar tiempo para que Cloudflared genere la URL pública
time.sleep(8)

# 6) Leer el log y extraer la URL
url = None
with open("cloudflared.log") as f:
    for line in f:
        m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
        if m:
            url = m.group(0)
            break

# 7) Mostrar el resultado al usuario
if url:
    print("✅ Tu dashboard está disponible aquí:\n", url)
else:
    print("⚠️ No se pudo obtener la URL. Verifica 'cloudflared.log' para más detalles.")


(Reading database ... 126285 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.7.0) over (2025.7.0) ...
Setting up cloudflared (2025.7.0) ...
Processing triggers for man-db (2.10.2-1) ...
✅ Tu dashboard está disponible aquí:
 https://sisters-est-cambridge-bathrooms.trycloudflare.com
